<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/Simple_Unet_Wetlands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import glob
import gdown
import zipfile
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import rasterio
import rasterio.plot
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import load_model
from keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
from scipy.ndimage import generic_filter
from scipy.stats import mode


In [17]:
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [4]:
directory_path = '/content/drive/MyDrive/datahub_bucket/Extract_TrainingData'
os.makedirs(directory_path, exist_ok=True)

In [5]:
# zip_files = glob.glob('/content/drive/MyDrive/datahub_bucket/TrainingData/*.zip')
# extract_dir = '/content/drive/MyDrive/datahub_bucket/Extract_TrainingData' # destination directory
# for zip_file in zip_files:
#     base_name = os.path.basename(zip_file)[:-4]
#     unzip_dir = os.path.join(extract_dir, base_name)
#     os.makedirs(unzip_dir, exist_ok=True)
#     with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#         zip_ref.extractall(unzip_dir)

In [7]:
extract_dir = '/content/drive/MyDrive/datahub_bucket/Extract_TrainingData'

In [8]:
csv_files = glob.glob('/content/drive/MyDrive/datahub_bucket/TrainingData/*.csv')
csv_list = []
for csv_file in csv_files:
    base_name = os.path.basename(csv_file)[:-4]
    csv_dir = os.path.join(extract_dir, base_name)
    base_csv = pd.read_csv(csv_file)
    base_csv['subset'] = base_name
    csv_list.append(base_csv)
    index_csv = pd.concat(csv_list, ignore_index=True)

In [9]:
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).size().reset_index(name='Count')
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).agg({'percent': 'mean'}).reset_index()

In [10]:
pivot_df = unique_rows_df.pivot(index=['tif_name', 'subset'], columns='label', values='percent').reset_index().fillna(0)

In [11]:
merged_df = index_csv.merge(pivot_df, on=['tif_name', 'subset'], how='left')
merged_df.drop(columns=['label'], inplace=True)

In [12]:
sum_df = merged_df.drop_duplicates(subset=['tif_name', 'subset'])

In [13]:
total_rows = len(sum_df)
train_fraction = 0.9 # modify this to set the training percentage
train_rows = int(total_rows * train_fraction)
val_rows = total_rows - train_rows

random_assignment = np.array([0] * train_rows + [1] * val_rows)
np.random.seed(42)
np.random.shuffle(random_assignment)
sum_df['random_split'] = random_assignment

<ipython-input-13-66bd3969d95d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['random_split'] = random_assignment


In [14]:
agg_df = sum_df.groupby(['random_split']).agg({13: 'mean',14:'mean',15: 'mean'}).reset_index()
print(agg_df)

   random_split        13        14        15
0             0  0.228634  0.045680  0.003364
1             1  0.341195  0.124668  0.007790


In [15]:
sum_df['Images_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Image_Subset/"+sum_df['tif_name']
sum_df['Labels_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Label_Subset/"+sum_df['tif_name']

<ipython-input-15-acfe803626d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Images_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Image_Subset/"+sum_df['tif_name']
<ipython-input-15-acfe803626d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Labels_path']="/content/drive/MyDrive/datahub_bucket/Extract_TrainingData/Maui_oneYear_Subset/Maui_Label_Subset/"+sum_df['tif_name']


In [16]:
sum_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,percent,tif_name,subset,13,14,15,random_split,Images_path,Labels_path
0,5121,2049,4097,1025,37,749157.194721,751614.766259,2.322698e+06,2.325155e+06,0.002741,4097_1025.tif,Maui_oneYear_subset,0.002741,0.009592,0.006812,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
3,2049,3073,1025,2049,66,741784.411875,744242.028903,2.320240e+06,2.322698e+06,0.001051,1025_2049.tif,Maui_oneYear_subset,0.000000,0.000000,0.001051,1,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
4,4097,3073,3073,2049,68,746699.579012,749157.145333,2.320240e+06,2.322698e+06,0.096800,3073_2049.tif,Maui_oneYear_subset,0.096800,0.099092,0.007901,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
7,5121,3073,4097,2049,69,749157.176045,751614.789917,2.320240e+06,2.322698e+06,0.046784,4097_2049.tif,Maui_oneYear_subset,0.046784,0.035456,0.019820,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
10,6145,3073,5121,2049,70,751614.801963,754072.422910,2.320240e+06,2.322698e+06,0.082713,5121_2049.tif,Maui_oneYear_subset,0.082713,0.089973,0.017363,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,28673,16385,27649,15361,508,805681.972014,808139.563877,2.288291e+06,2.290749e+06,0.034338,27649_15361.tif,Maui_oneYear_subset,0.034338,0.035327,0.000000,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
269,25601,17409,24577,16385,537,798309.191371,800766.782820,2.285834e+06,2.288291e+06,0.185758,24577_16385.tif,Maui_oneYear_subset,0.185758,0.000000,0.000000,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
270,26625,17409,25601,16385,538,800766.792034,803224.352420,2.285834e+06,2.288291e+06,0.462209,25601_16385.tif,Maui_oneYear_subset,0.462209,0.000000,0.000174,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...
272,27649,17409,26625,16385,539,803224.350722,805681.983466,2.285834e+06,2.288291e+06,0.003212,26625_16385.tif,Maui_oneYear_subset,0.003212,0.000000,0.000000,0,/content/drive/MyDrive/datahub_bucket/Extract_...,/content/drive/MyDrive/datahub_bucket/Extract_...


In [33]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization

In [61]:
KERNEL_SIZE = 512
BANDS = range(7)
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'
METRICS = ['categorical_accuracy']

In [62]:
def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('LeakyReLU')(encoder) # from relu to LeakyReLU
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('LeakyReLU')(encoder) # from relu to LeakyReLU
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	return decoder

def get_model():
	inputs = layers.Input(shape=[KERNEL_SIZE, KERNEL_SIZE, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(25, (1, 1), activation='softmax')(decoder0) # kept because this is a multiclass classification

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

In [68]:
class DataGenerator(Sequence):
    def __init__(self, image_files, label_files, img_height, img_width, batch_size, num_classes):
        self.image_files = image_files
        self.label_files = label_files
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        self.num_classes = num_classes
        # self.noise = noise

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.image_files[index * self.batch_size : (index + 1) * self.batch_size]
        batch_images, batch_labels = self.load_images_and_labels(batch_files)
        return batch_images, batch_labels

    def load_and_reshape_image(self, image_path):
        with rasterio.open(image_path) as src:
            image = src.read()
            image = image.transpose((1, 2, 0))
            if image.shape[0] != self.img_height or image.shape[1] != self.img_width:
                image = cv2.resize(image, (self.img_width, self.img_height), interpolation=cv2.INTER_NEAREST)
            if len(image.shape) == 3 and image.shape[2] == 1:
                image = np.squeeze(image, axis=2)
            # if self.noise:
            #     variance = np.var(image)
            #     image = image*random_noise(image, mode='gaussian', var=variance)
            return image

    def load_images_and_labels(self, image_files):
        images = []
        labels = []

        for image_file in image_files:
            image = self.load_and_reshape_image(image_file)
            images.append(image)

        for image_file in image_files:
            label_file = image_file.replace("Images", "Labels")
            label = self.load_and_reshape_image(label_file)
            label -= 1
            label = to_categorical(label, num_classes=self.num_classes)[0]
            labels.append(label)

        return np.array(images), np.array(labels)

In [69]:
model = get_model()

In [65]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [72]:
training_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==0], sum_df['Labels_path'][sum_df['random_split']==0], 512, 512, 4, 5)
validation_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==1], sum_df['Labels_path'][sum_df['random_split']==1], 512, 512, 4, 5)

In [73]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','categorical_accuracy'])
callbacks = [
    ModelCheckpoint("wetlands.h5", save_best_only=True)
]

model.fit(training_data_generator, validation_data=validation_data_generator, epochs=5, callbacks=callbacks,shuffle=True)

IndexError: ignored